In [1]:
import pandas as pd
from gensim.models import Word2Vec
import nltk
import re
from nltk import word_tokenize, pos_tag
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import enchant
import numpy as np
from collections import Counter
import seaborn as sns


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import random
from sklearn.model_selection import KFold



from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

import emoji
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords



#nltk.download('punkt_tab')
#nltk.download("words")
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger_eng')
#%matplotlib qt
#nltk.download('stopwords')
%matplotlib inline


In [3]:
df = pd.read_csv("scitweets_export.tsv", sep = "\t")

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline


#X_train, X_test, y_train, y_test = train_test_split(list(df["text"]), list(df["science_related"]), test_size=0.2, random_state=42)

# Création du pipeline
pipeline = ImbPipeline([
    ('vectorizer', TfidfVectorizer()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(class_weight='balanced', random_state=42))
])

scores = cross_val_score(pipeline, list(df["text"]), list(df["science_related"]), cv=10, scoring='accuracy')
print("accuracy:", scores)
print("Moyenne:", scores.mean())
print("---------------------------------------------------------------------------------\n")
scores = cross_val_score(pipeline, list(df["text"]), list(df["science_related"]), cv=10, scoring='f1')
print("F1 scores:", scores)
print("Moyenne:", scores.mean())

accuracy: [0.71929825 0.73684211 0.83333333 0.79824561 0.76315789 0.75438596
 0.83333333 0.81578947 0.78070175 0.8245614 ]
Moyenne: 0.7859649122807018
---------------------------------------------------------------------------------

F1 scores: [0.30434783 0.34782609 0.6779661  0.61016949 0.52631579 0.51724138
 0.74666667 0.69565217 0.62686567 0.6875    ]
Moyenne: 0.5740551187269347
